Importance sampling can be applied to Markov chains, when it is often called fast simulation. Suppose we have a discrete-time Markov chain with jump probabilities $P_{ij}$. The path which the Markov chain takes can be regarded as a random variable in the space of sample paths, and we might be interested in the probability that the path is in some particular set of paths.

Consider another Markov chain with twisted jump probabilities $P'_{ij}$ such that $P'_{ij} > 0$ whenever $P_{ij} > 0$. Let $x_0 \to x_1 \to \cdots \to x_n$ be an observed path which we shall denote as $S$. The probability of $S$ in a Markov chain, denoted $\pi(S) is the product of the transition probabilities for each step, given a starting state.

Assuming the chain starts at a fixed state $x_0$, the probability of the path is
\begin{equation}
    \pi(S) = \Pr(X_1=x_1 \vert X_0=x_0) \cdots \Pr(X_n=x_n \vert X_{n-1}=x_{n-1}).
\end{equation}
This can be written as the product of the jump probabilities for each transition in the path
\begin{equation}
    \pi(S) = P_{0,1} \cdots P_{n-1,n} = \prod_{k=1}^n P_{k-1,k}.
\end{equation}

Similarly, the probability of observing the same path $S$ under the twisted Markov chain with jump probabilities $P'_{ij}$ shall be denoted by $\pi'(S)$ and is given by
\begin{equation}
    \pi'(S) = \prod_{k=1}^n P'_{k-1, k}.
\end{equation}

The likelihood ratio $L$ is therefore
\begin{equation}
    L = \frac{\pi(S)}{\pi'(S)} = \frac{\prod_{k=1}^n P_{k-1, k}}{\prod_{k=1}^n P'_{k-1,k}} = \prod_{k=1}^n\frac{P_{k-1, k}}{P'_{k-1,k}}.
\end{equation}
This means the likelihood ratio for the entire path is simply the product of the likelihood ratios of each individual jump taken along that path.


---

Consider a simple random walk $(X_n)_{n>0}$ on the non-negative integers with up probability $p < 1/2$ and down probability $1 - p$, except up probability $1$ when the walk is at $0$. We are interested in the event that, on a single excursion away from $0$, the random walk hits some high level $C$ before it returns to $0$.

For instance, consider $p = 1/4$ and $C = 30$ and find the probability $\Pr(T_C < T_0 \vert X_0 = 0)$, where $T_x = \inf\{n \geq 1 : X_n = x\}$. The first step is deterministic $X_1 = 1$, so starting from $0$ is equivalent to starting from $1$

This is a standard gambler's ruin problem with the following parameters:
*   The gambler's starting fortune (initial state) is $i = 1$.
*   The target fortune to win (absorbing upper barrier) is $N = C = 30$.
*   The point of ruin (absorbing lower barrier) is $0$.
*   The probability of moving up (winning a bet) is $p = 1/4$.
*   The probability of moving down (losing a bet) is $q = 1 - p = 3/4$.
The formula for the probability of reaching state $N$ before state $0$, starting from state $i$, when $p \neq q$ is
\begin{equation}
    \Pr(\text{reach $N$ before $0$ } \vert \text{ start at $i$}) = \frac{1 - (q/p)^i}{1 - (q/p)^N} = \frac{2}{3^{30}-1}.
\end{equation}
This is a very small number so we consider a fast simulation using another
random walk with up probability $p' > 1/2$ and down probability $1 - p'$.


The same calculation shows that for a single excursion from $0$, the probability that a twisted random walk reaches $C$ is
\begin{equation}
    P'(T_C < T_0 | X_1 = 1) = \frac{1 - (q'/p')}{ 1 - (q'/p')^C}.
\end{equation}

For the original random walk, as $C \to \infty$, since $q/p > 1$, the probability decays exponentially to $0$. For the twisted random walk, $q'/p' < 1$ instead, so the probability converges to $1-(q'/p') > 0$

Simulating the original random walk is highly inefficient since the event of reaching $C=30$ is very rare, we would need to run a large number of simulations to observe the event. Most simulated paths would quickly return to $0$, providing no information. The twisted random walk has a strong upward drift. When we simulate this process, the event of reaching $C$ is no longer rare. This means that a large fraction of our simulated paths will be successful paths that reach $C$.

We can then use the likelihood ratio to re-weight the outcome of each simulation. Each successful path will be multiplied by a very small likelihood ratio, correctly down-weighting it to reflect its true probability in the original system.

In [3]:
import numpy as np
import math

def run_fast_simulation(p_prime, num_simulations=10000):
    '''
    Estimates P(T_C < T_0) for a random walk using importance sampling.
    Args:
        p_prime: The up probability for the twisted random walk.
        num_simulations: The number of paths to simulate.
    Returns:
        A tuple containing the estimated probability and the variance of the estimates.
    '''
    # Constants for the original random walk
    p_original = 1/4.0
    q_original = 1 - p_original
    C = 30
    # Setup for the twisted random walk
    q_prime = 1 - p_prime
    estimates = []

    for _ in range(num_simulations):
        current_pos = 1
        num_up_steps = 0
        num_down_steps = 0

        while 0 < current_pos < C:
            # Simulate one step using the twisted probability p'
            if np.random.uniform(0, 1) < p_prime:
                current_pos += 1
                num_up_steps += 1
            else:
                current_pos -= 1
                num_down_steps += 1

        # Check if the path reached C
        if current_pos == C:
            # Calculate the likelihood ratio for this successful path
            log_L = num_up_steps * math.log(p_original / p_prime) + \
                    num_down_steps * math.log(q_original / q_prime)
            L = math.exp(log_L)
            estimates.append(L)
        else:
            # Path hit 0, contribution is 0
            estimates.append(0)

    # The final estimate is the mean of all individual outcomes
    estimated_prob = np.mean(estimates)
    variance = np.var(estimates)
    return estimated_prob, variance

# The true probability
true_prob = 2 / (3**30 - 1)

p_prime_values = [0.60, 0.70, 0.75, 0.80, 0.90]

print(f"True Probability ≈ {true_prob:.6e}\n")
print(f"{'p_prime':<10} | {'Estimated Probability':<25} | {'Variance':<15}")
print("-" * 55)

for p_p in p_prime_values:
    est_prob, var = run_fast_simulation(p_p)
    print(f"{p_p:<10.2f} | {est_prob:<25.6e} | {var:<15.6e}")

True Probability ≈ 9.713871e-15

p_prime    | Estimated Probability     | Variance       
-------------------------------------------------------
0.60       | 8.748887e-15              | 5.764820e-27   
0.70       | 9.868965e-15              | 1.887295e-28   
0.75       | 9.750784e-15              | 4.699900e-29   
0.80       | 9.626804e-15              | 2.369591e-28   
0.90       | 7.057914e-15              | 1.133757e-26   


For every choice of $p' > 1/2$, the estimated probability is relatively close to the true analytical value, demonstrating the estimator is unbiased. The quality of an estimator is measured by its variance.

The value $p' = 3/4$ minimises this variance. It is equal to $1 - p$, the down probability. This is a known result for the gambler's ruin problem, where the optimal importance sampling distribution is found by reflecting the original probabilities.

The estimator $\hat{\gamma}$ is the outcome of a single simulation trail and is a discrete random variable which takes two possible values. If the path hits $0$, then $\hat{\gamma}$ is $0$, otherwise if the path hits $C$ then $\hat{\gamma}$ is the likelihood ratio $L$. For any path that starts at $1$ and reaches $C=30$, let $n_{+}$ be the number of up steps and $n_-$ be the number of down steps. The net displacement is $n_+ - n_- = 29$. The likelihood ratio for such a path is
\begin{equation}
L = (p/p')^{n_+} (q/q')^{n_-} = (p/q)^{n_+} (q/p)^{n_-} = (p/q)^{n_+ - n_-} = (1/3)^{29}.
\end{equation}
Thus, for the optimal choice of $p'$, the likelihood ratio is a constant value for every successful path, independent of the length of the path.

We now use the gambler's ruin formula with the twisted probabilities
\begin{equation}
\Pr(\hat{\gamma} = L) = \frac{1 - (q'/p')}{1 - (q'/p')^C} = \frac{1 - (p/q)}{1 - (p/q)^30} = \frac{2/3}{1-(1/3)^{30}}.
\end{equation}

To determine how close the average of $N=10,000$ trials will be to the true value, we calculate the standard error of the mean estimate. Let $\bar{\gamma}$ be the sample mean. By the central limit theorem, $\bar{\gamma}$ will be approximately normally distributed around the true value $\gamma$. The standard deviation of this distribution is the standard error $SE = \sigma/\sqrt{N}$, where $\sigma^2 = Var(\hat{\gamma})$.
*   $Var(\hat{\gamma}) = E[\hat{\gamma}^2] - E[\hat{\gamma}]^2$;
*   $E[\hat{\gamma}]$ is the true probability $\gamma = 2 / (3^{30} - 1)$.
*   $E[\hat{\gamma}^2] = (1/3)^{58} ( (2/3) / (1 - (1/3)^{30}) ) = (2/3^{29}) / (3^{30} - 1)$.
Numerically, $Var(\hat{\gamma}) \approx = 4.734 \cdot 10⁻^{-31}$, so $\sigma \approx 6.88 \cdot 10^{-16}$. Thus,
\begin{equation}
    \mathop{SE} = \frac{\sigma}{\sqrt{N}} \approx 6.88 x 10^{-18}.
\end{equation}
The relative error is $\mathop{SE}/\gamma \approx 0.007$. This means that after 10,000 trials, we would expect our estimate to be within less than $1\%$ of the true value itself.